<a href="https://colab.research.google.com/github/DavideRutigliano/frigate-models/blob/main/notebooks/YOLOv8_Pretrained_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
%%capture
!pip install ultralytics

## Select model

In [45]:
from ultralytics import YOLO

model_name = "yolo11s" #@param ["yolov8s", "yolov8m", "yolov8l", "yolov9s", "yolov9m", "yolov9c", "yolo11s", "yolo11m", "yolo11l"]
input_width = 320 #@param {type:"slider", min:320, max:640, step:320}
input_height = 320 #@param {type:"slider", min:320, max:640, step:320}
quantization = "fp16"

MODEL_FILENAME = f"{model_name}_{quantization}_{input_width}x{input_height}.onnx"

In [46]:
model = YOLO(f"{model_name}.pt")
model.export(
  format="onnx",
  imgsz=[input_height,input_width],
  half=(quantization=="fp16"),
  optimize=False,
)
!mv {model_name}.onnx {MODEL_FILENAME}

100%|██████████| 18.4M/18.4M [00:00<00:00, 151MB/s]

Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.30GHz)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0


YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs

PyTorch: starting from 'yolo11s.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 84, 2100) (18.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 2.1s, saved as 'yolo11s.onnx' (36.1 MB)

Export complete (2.8s)
Results saved to /content
Predict:         yolo predict task=detect model=yolo11s.onnx imgsz=320  
Validate:        yolo val task=detect model=yolo11s.onnx imgsz=320 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


In [47]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.float32)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"images": dummy_input})

[array([[[     6.3959,      13.846,      22.053, ...,      166.86,      198.92,      233.55],
         [     2.2839,      2.3691,      2.6521, ...,      263.13,      255.86,      238.57],
         [       6.98,      25.196,      26.798, ...,      349.95,      247.38,       179.3],
         ...,
         [          0,           0,           0, ...,  5.9605e-08,  1.0341e-05,  3.6746e-05],
         [          0,           0,           0, ...,  3.8743e-07,  3.5763e-06,  8.5533e-06],
         [          0,           0,           0, ...,  3.5763e-07,  4.3809e-06,  7.0632e-06]]], dtype=float32)]

## Download model

In [48]:
%cd /content
from google.colab import files

files.download(MODEL_FILENAME)

/content


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>